In [32]:
import argparse
import logging
import os

import numpy as np
import torch
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
from train import train, train_and_evaluate

import utils
import model.net as net
from  model.data_loader import  fetch_dataloader
from evaluate import evaluate

We have 2 files with training script: `train.py` and `evaluate.py`. Let's start with the first one.

## `train.py`

This file contains 3 functions: `main`, `train` and `train_and_evaluate`.

### main

#### get `params`

We just load `params.json` file from the specified model.

In [16]:
model_dir = 'experiments/base_model'
json_path = os.path.join(model_dir, 'params.json')
params = utils.Params(json_path)

In [17]:
params.dict

{'learning_rate': 0.001,
 'batch_size': 1,
 'num_epochs': 1,
 'dropout_rate': 0.8,
 'num_channels': 32,
 'save_summary_steps': 100,
 'num_workers': 0,
 'cuda': 0}

In [18]:
torch.manual_seed(230);

#### get `dataloaders`

We have to provide only `types`, data directory is included in dataloader code.

In [33]:
dataloaders = fetch_dataloader(['train', 'val'], params)
train_dl = dataloaders['train']
val_dl = dataloaders['val']

In [34]:
len(train_dl), len(val_dl)

(25, 5)

In [35]:
img, target = next(iter(train_dl))

In [38]:
img.shape, target.shape

(torch.Size([1, 1, 512, 512]), torch.Size([1, 1, 512, 512]))

#### get model and everything else

There are 2 questions here:

- how do we choose our loss; and
- how do we compute accuracy;

We use `BCEWithLogitsLoss`. This is the standard cross-entropy loss but combined with sigmoid:

> This loss combines a Sigmoid layer and the BCELoss in one single class. This version is more numerically stable than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the log-sum-exp trick for numerical stability.

It turns out that `isbi` uses much more sofisticated metrics so using accuracy is just the first step for us. To compute it we have to convert our outputs into matrix of `0` and `1`.

In [37]:
model = net.Unet()
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
loss_fn = net.loss_fn
metrics = net.metrics

Let's now run `train` for 1 epoch.

In [39]:
params.num_epochs = 1

In [40]:
train(model, optimizer, loss_fn, train_dl, metrics, params)

100%|██████████| 25/25 [04:10<00:00, 10.27s/it, loss=0.490]
- Train metrics: accuracy: 0.000 ; loss: 0.777
